# 2.2.1 Basic Connectivity

<div style="background-color: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; padding: 10px; margin: 10px;">
<strong>📋 Workshop Contents</strong>
<ul style="line-height: 1.2;">
<li><a href="#What-You'll-Learn">What You'll Learn</a></li>
<li><a href="#Prerequisites">Prerequisites</a></li>
<li><a href="#1-Setting-Up-Environment">1. Setting Up Environment</a></li>
<li><a href="#2-Get-Connection-Details-from-Secrets-Manager">2. Get Connection Details from Secrets Manager</a></li>
<li><a href="#3-Testing-Database-Connectivity">3. Testing Database Connectivity</a></li>
<li><a href="#4-Connection-Diagnostics">4. Connection Diagnostics</a></li>
<li><a href="#Next-Steps">Next Steps</a></li>
</ul>
</div>

Welcome to the first part of connecting to Aurora PostgreSQL! In this notebook, we'll establish basic connectivity using different methods. 🚀

## What You'll Learn
- Retrieve credentials from AWS Secrets Manager
- Set up connection environment variables
- Connect using psql CLI
- Connect using Python (psycopg)
- Connect using AWS CLI
- Troubleshoot common connection issues

## Prerequisites
- You have set up the infrastructure and launched Aurora cluster followed by [2.1 Creating Your First Aurora Cluster](../2.1_Crearting_Your_First_Aurora_Cluster/README.MD). 
- Installed tools:
  - `psql` client
  - Python 3.7+
  - AWS CLI
  - `jq` for JSON parsing

> 🔑 **Security Note**: This notebook retrieves credentials from AWS Secrets Manager for improved security.

Let's get started by setting up our environment!

## 1. Setting Up Environment

Install essential Python packages needed for Aurora PostgreSQL connectivity including psycopg for database connections, boto3 for AWS SDK operations, and pandas for data manipulation. The installation runs quietly with a completion message once all packages are successfully installed.

In [ ]:
%%bash
echo "Checking package availability..."
conda list | grep -E "(boto3|pandas|psycopg)" || echo "Some packages may need installation"

Install packages if needed.

In [ ]:
%%bash
pip install boto3 psycopg[binary] pandas 

### Check VPC Configuration

Analyze the network configuration between your SageMaker notebook instance and Aurora cluster to identify potential connectivity issues. The script compares VPC IDs, checks security group rules, and automatically adds necessary access rules if missing with detailed VPC information and setup confirmation.

In [ ]:
%%bash
# Detect cluster name and setup type
# First try to find CloudFormation-created clusters (aurora-sv2 pattern) - get most recent
AURORA_CLUSTER=$(aws rds describe-db-clusters --query 'DBClusters[?contains(DBClusterIdentifier, `aurora-sv2`)] | sort_by(@, &ClusterCreateTime) | [-1].DBClusterIdentifier' --output text)

# If no CloudFormation cluster found, try manual setup pattern
if [ -z "$AURORA_CLUSTER" ] || [ "$AURORA_CLUSTER" == "None" ]; then
    AURORA_CLUSTER=$(aws rds describe-db-clusters --query 'DBClusters[?DBClusterIdentifier==`aurora-demo`].DBClusterIdentifier' --output text | head -1)
fi

# If still no cluster found, try to find any Aurora PostgreSQL cluster - get most recent
if [ -z "$AURORA_CLUSTER" ] || [ "$AURORA_CLUSTER" == "None" ]; then
    AURORA_CLUSTER=$(aws rds describe-db-clusters --query 'DBClusters[?Engine==`aurora-postgresql`] | sort_by(@, &ClusterCreateTime) | [-1].DBClusterIdentifier' --output text)
    if [ -n "$AURORA_CLUSTER" ] && [ "$AURORA_CLUSTER" != "None" ]; then
        echo "⚠️ Found Aurora PostgreSQL cluster with different naming: $AURORA_CLUSTER"
    fi
fi

if [ -z "$AURORA_CLUSTER" ]; then
    echo "❌ No Aurora cluster found. Please ensure you have created an Aurora cluster using either:"
    echo "   - CloudFormation template (aurora-complete-stack.yml)"
    echo "   - Manual setup (2.1.2 notebook)"
    echo ""
    echo "Available clusters:"
    aws rds describe-db-clusters --query 'DBClusters[*].{Identifier:DBClusterIdentifier,Engine:Engine,Status:Status}' --output table
    exit 0
fi

echo "Found Aurora cluster: $AURORA_CLUSTER"

# Determine setup type and export to temp file
if [[ "$AURORA_CLUSTER" == *"aurora-sv2"* ]]; then
    SETUP_TYPE="cloudformation"
    echo "Setup type: CloudFormation (aurora-complete-stack.yml)"
elif [[ "$AURORA_CLUSTER" == "aurora-demo" ]]; then
    SETUP_TYPE="manual"
    echo "Setup type: Manual (Jupyter notebook)"
else
    SETUP_TYPE="unknown"
    echo "Setup type: Unknown (will attempt to detect secret)"
fi

# Get SageMaker notebook instance name
NOTEBOOK_NAME=$(cat /opt/ml/metadata/resource-metadata.json 2>/dev/null | jq -r '.ResourceName')
echo "SageMaker notebook name: $NOTEBOOK_NAME"

# Check if notebook name is null (manual VPC setup)
if [ "$NOTEBOOK_NAME" == "null" ] || [ -z "$NOTEBOOK_NAME" ]; then
    echo "⚠️ SageMaker VPC does not exist. VPC must be created manually."
    echo "Skipping VPC configuration check."
    
    # Export essential variables for later use
    echo "export AURORA_CLUSTER=$AURORA_CLUSTER" > .aurora_env
    echo "export SETUP_TYPE=$SETUP_TYPE" >> .aurora_env
    echo "export NOTEBOOK_NAME=$NOTEBOOK_NAME" >> .aurora_env
    
    exit 0
fi

# Get SageMaker notebook VPC and security group
echo "Getting SageMaker notebook VPC information..."
if ! SAGEMAKER_INFO=$(aws sagemaker describe-notebook-instance --notebook-instance-name $NOTEBOOK_NAME 2>/dev/null); then
    echo "⚠️ Could not retrieve SageMaker notebook instance info. Trying alternative method..."
    # Try to get current instance metadata
    INSTANCE_ID=$(curl -s http://169.254.169.254/latest/meta-data/instance-id 2>/dev/null || echo "unknown")
    if [ "$INSTANCE_ID" != "unknown" ]; then
        SAGEMAKER_SG=$(aws ec2 describe-instances --instance-ids $INSTANCE_ID --query 'Reservations[0].Instances[0].SecurityGroups[0].GroupId' --output text 2>/dev/null || echo "")
        SAGEMAKER_SUBNET=$(aws ec2 describe-instances --instance-ids $INSTANCE_ID --query 'Reservations[0].Instances[0].SubnetId' --output text 2>/dev/null || echo "")
    else
        echo "❌ Cannot determine SageMaker instance details. Skipping VPC check."
        exit 0
    fi
else
    SAGEMAKER_SUBNET=$(echo $SAGEMAKER_INFO | jq -r '.SubnetId')
    SAGEMAKER_SG=$(echo $SAGEMAKER_INFO | jq -r '.SecurityGroups[0]')
fi

if [ -n "$SAGEMAKER_SUBNET" ] && [ "$SAGEMAKER_SUBNET" != "null" ]; then
    SAGEMAKER_VPC=$(aws ec2 describe-subnets --subnet-ids $SAGEMAKER_SUBNET | jq -r '.Subnets[0].VpcId')
else
    echo "❌ Cannot determine SageMaker subnet. Skipping VPC check."
    exit 0
fi

# Get Aurora cluster VPC and security group
echo "Getting Aurora cluster VPC information..."
AURORA_SG=$(aws rds describe-db-clusters --db-cluster-identifier $AURORA_CLUSTER | jq -r '.DBClusters[0].VpcSecurityGroups[0].VpcSecurityGroupId')
AURORA_VPC=$(aws ec2 describe-security-groups --group-ids $AURORA_SG | jq -r '.SecurityGroups[0].VpcId')

# Export all info to temp file for use in other cells
echo "export AURORA_CLUSTER=$AURORA_CLUSTER" > .aurora_env
echo "export SETUP_TYPE=$SETUP_TYPE" >> .aurora_env
echo "export NOTEBOOK_NAME=$NOTEBOOK_NAME" >> .aurora_env
echo "export SAGEMAKER_VPC=$SAGEMAKER_VPC" >> .aurora_env
echo "export SAGEMAKER_SG=$SAGEMAKER_SG" >> .aurora_env
echo "export AURORA_VPC=$AURORA_VPC" >> .aurora_env
echo "export AURORA_SG=$AURORA_SG" >> .aurora_env

echo "SageMaker notebook VPC: $SAGEMAKER_VPC"
echo "SageMaker notebook security group: $SAGEMAKER_SG"
echo "Aurora cluster VPC: $AURORA_VPC"
echo "Aurora cluster security group: $AURORA_SG"
echo "✅ All environment information saved for later use"

# Compare VPCs
echo "Checking VPC configuration..."
if [ "$SAGEMAKER_VPC" == "$AURORA_VPC" ]; then
    echo "✅ SageMaker notebook and Aurora cluster are in the same VPC. No VPC peering needed."
else
    echo "⚠️ SageMaker notebook and Aurora cluster are in different VPCs. VPC peering required. See the next section."
fi

# Check if Aurora security group allows access from SageMaker security group
if [ -n "$SAGEMAKER_SG" ] && [ "$SAGEMAKER_SG" != "null" ]; then
    echo "Checking Aurora security group rules..."
    
    # Use describe-security-groups (more compatible than describe-security-group-rules)
    if SG_RULES=$(aws ec2 describe-security-groups \
                    --group-ids $AURORA_SG \
                    --query 'SecurityGroups[0].IpPermissions[?FromPort==`5432`]' \
                    --output json 2>/dev/null); then
        
        # Check if SageMaker SG is referenced in any rule
        if echo "$SG_RULES" | jq -e ".[].UserIdGroupPairs[]? | select(.GroupId==\"$SAGEMAKER_SG\")" >/dev/null 2>&1; then
            echo "✅ Aurora security group already allows access from SageMaker security group."
        else
            echo "⚠️ Aurora security group does not allow access from SageMaker security group."
            echo "Adding security group rule to allow SageMaker access..."
            
            # Add security group rule
            if aws ec2 authorize-security-group-ingress \
                --group-id $AURORA_SG \
                --protocol tcp \
                --port 5432 \
                --source-group $SAGEMAKER_SG \
                --description "Allow access from SageMaker notebook" 2>/dev/null; then
                echo "✅ Security group rule added successfully."
            else
                echo "⚠️ Failed to add security group rule. It may already exist or you may lack permissions."
            fi
        fi
    else
        echo "⚠️ Could not retrieve security group details. Attempting to add rule anyway..."
        
        # Try to add the rule - if it exists, it will fail gracefully
        if aws ec2 authorize-security-group-ingress \
            --group-id $AURORA_SG \
            --protocol tcp \
            --port 5432 \
            --source-group $SAGEMAKER_SG \
            --description "Allow access from SageMaker notebook" 2>/dev/null; then
            echo "✅ Security group rule added successfully."
        else
            echo "⚠️ Could not add security group rule. It may already exist or you may lack permissions."
        fi
    fi
else
    echo "⚠️ SageMaker security group not found. Skipping security group rule check."
fi

### Create VPC Peering (if needed)

If your SageMaker notebook and Aurora cluster are in different VPCs, use these commands to create VPC peering:

In [ ]:
%%bash
# Source all environment variables
source .aurora_env

echo "Using saved VPC information:"
echo "SageMaker VPC: $SAGEMAKER_VPC"
echo "Aurora VPC: $AURORA_VPC"

# Only run this if the VPCs are different
if [ "$SAGEMAKER_VPC" != "$AURORA_VPC" ]; then
    echo "Creating VPC peering connection..."
    
    # Create VPC peering connection
    PEERING_ID=$(aws ec2 create-vpc-peering-connection \
                --vpc-id $SAGEMAKER_VPC \
                --peer-vpc-id $AURORA_VPC \
                --query 'VpcPeeringConnection.VpcPeeringConnectionId' \
                --output text)
    
    echo "VPC peering connection created: $PEERING_ID"
    
    # Accept the peering connection
    aws ec2 accept-vpc-peering-connection --vpc-peering-connection-id $PEERING_ID
    echo "VPC peering connection accepted"
    
    # Get CIDR blocks
    SAGEMAKER_CIDR=$(aws ec2 describe-vpcs --vpc-ids $SAGEMAKER_VPC \
                    --query 'Vpcs[0].CidrBlock' --output text)
    AURORA_CIDR=$(aws ec2 describe-vpcs --vpc-ids $AURORA_VPC \
                --query 'Vpcs[0].CidrBlock' --output text)
    
    # Update SageMaker route tables
    echo "Updating SageMaker route tables..."
    SAGEMAKER_RTBS=$(aws ec2 describe-route-tables \
                    --filters Name=vpc-id,Values=$SAGEMAKER_VPC \
                    --query 'RouteTables[*].RouteTableId' --output text)
    
    for RTB in $SAGEMAKER_RTBS; do
        aws ec2 create-route \
            --route-table-id $RTB \
            --destination-cidr-block $AURORA_CIDR \
            --vpc-peering-connection-id $PEERING_ID
        echo "Added route to $RTB"
    done
    
    # Update Aurora route tables
    echo "Updating Aurora route tables..."
    AURORA_RTBS=$(aws ec2 describe-route-tables \
                --filters Name=vpc-id,Values=$AURORA_VPC \
                --query 'RouteTables[*].RouteTableId' --output text)
    
    for RTB in $AURORA_RTBS; do
        aws ec2 create-route \
            --route-table-id $RTB \
            --destination-cidr-block $SAGEMAKER_CIDR \
            --vpc-peering-connection-id $PEERING_ID
        echo "Added route to $RTB"
    done
    
    echo "✅ VPC peering setup complete!"
else
    echo "SageMaker and Aurora are in the same VPC. No peering needed."
fi

### Get Connection Details from Secrets Manager

Map the Aurora cluster to its associated secret and export the mapping to a temp file for later use. This section identifies the correct secret name based on the setup type without storing actual credentials.

In [ ]:
%%bash
# Source cluster info
source .aurora_env

echo "Mapping Aurora cluster to secret..."
echo "Cluster: $AURORA_CLUSTER"
echo "Setup type: $SETUP_TYPE"

# Detect secret name based on setup type
if [ "$SETUP_TYPE" == "cloudformation" ]; then
    # CloudFormation setup - find stack by ClusterEndpoint containing 'aurora-sv2'
    echo "Finding CloudFormation stack with ClusterEndpoint containing 'aurora-sv2'..."
    
    CF_STACK_NAME=""
    for stack in $(aws cloudformation list-stacks --stack-status-filter CREATE_COMPLETE UPDATE_COMPLETE --query 'StackSummaries[*].StackName' --output text); do
        CLUSTER_ENDPOINT=$(aws cloudformation describe-stacks --stack-name $stack --query "Stacks[0].Outputs[?OutputKey=='ClusterEndpoint'].OutputValue" --output text 2>/dev/null)
        if [ ! -z "$CLUSTER_ENDPOINT" ] && [ "$CLUSTER_ENDPOINT" != "None" ] && [[ "$CLUSTER_ENDPOINT" == *"aurora-sv2"* ]]; then
            CF_STACK_NAME=$stack
            SECRET_NAME=$(aws cloudformation describe-stacks --stack-name $stack --query "Stacks[0].Outputs[?OutputKey=='SecretArn'].OutputValue" --output text)
            echo "Found CloudFormation stack: $CF_STACK_NAME"
            echo "Found secret ARN: $SECRET_NAME"
            echo "export CF_STACK_NAME=$CF_STACK_NAME" >> .aurora_env
            break
        fi
    done
    
    if [ -z "$CF_STACK_NAME" ]; then
        echo "❌ No CloudFormation stack found with ClusterEndpoint containing 'aurora-sv2'"
        exit 0
    fi
elif [ "$SETUP_TYPE" == "manual" ]; then
    # Manual setup - get managed secret directly from Aurora cluster
    echo "Getting managed secret from Aurora cluster..."
    MANAGED_SECRET_ARN=$(aws rds describe-db-clusters --db-cluster-identifier $AURORA_CLUSTER --query 'DBClusters[0].MasterUserSecret.SecretArn' --output text 2>/dev/null)
    if [ "$MANAGED_SECRET_ARN" != "None" ] && [ ! -z "$MANAGED_SECRET_ARN" ]; then
        SECRET_NAME=$MANAGED_SECRET_ARN
        echo "Found managed secret: $SECRET_NAME"
    else
        echo "❌ No managed secret found for cluster $AURORA_CLUSTER"
        exit 0
    fi
else
    # Unknown setup - ask user for secret name
    echo "Available secrets:"
    aws secretsmanager list-secrets --query 'SecretList[*].{Name:Name,Description:Description}' --output table
    echo ""
    echo "❓ Please provide the secret name for your Aurora cluster credentials."
    echo "You can find it in the AWS Secrets Manager console or from the list above."
    echo ""
    echo "To continue, run the following command with your secret name:"
    echo "export SECRET_NAME='your-secret-name-here'"
    echo "echo \"export SECRET_NAME=$SECRET_NAME >> .aurora_env\""
    exit 0
fi

if [ -z "$SECRET_NAME" ]; then
    echo "❌ No Aurora credentials secret found"
    exit 0
fi

echo "Using secret: $SECRET_NAME"

# Export secret mapping to temp file (no credentials stored)
echo "export SECRET_NAME=$SECRET_NAME" >> .aurora_env

echo "✅ Secret mapping saved for later use"

# Get cluster endpoint and export
CLUSTER_ENDPOINT=$(aws rds describe-db-clusters \
    --db-cluster-identifier $AURORA_CLUSTER \
    --query 'DBClusters[0].Endpoint' \
    --output text)

echo "Cluster endpoint: $CLUSTER_ENDPOINT"
echo "export CLUSTER_ENDPOINT=$CLUSTER_ENDPOINT" >> .aurora_env

## 3. Testing Database Connectivity

Now let's test different methods to connect to your Aurora PostgreSQL cluster. First, we'll retrieve the credentials from Secrets Manager, then test various connection methods.

### Method 1: Using psql CLI

Connect using the psql command-line client with the retrieved credentials.

In [ ]:
%%bash
# Source cluster and secret info
source .aurora_env

# Retrieve Credentials from Secrets Manager
echo "Retrieving credentials from secret: $SECRET_NAME"

# Get credentials from Secrets Manager
SECRET_VALUE=$(aws secretsmanager get-secret-value \
    --secret-id $SECRET_NAME \
    --query 'SecretString' \
    --output text)

# Parse JSON to extract username and password
export DB_USERNAME=$(echo $SECRET_VALUE | jq -r '.username')
export DB_PASSWORD=$(echo $SECRET_VALUE | jq -r '.password')

echo "Username: $DB_USERNAME"
echo "Password: [HIDDEN]"
echo "✅ Credentials retrieved and ready for connections"
echo "Testing connection with psql..."
echo "Connecting to: $CLUSTER_ENDPOINT"

# Connect using psql with retrieved credentials
psql "host=$CLUSTER_ENDPOINT port=5432 dbname=mylab user=$DB_USERNAME password=$DB_PASSWORD sslmode=require" \
    -c "SELECT version();" \
    -c "SELECT current_timestamp;"

# If successful, you'll see PostgreSQL version and current timestamp

### Method 2: Using Python with psycopg
Demonstrate programmatic database connectivity using psycopg with the credentials retreived from AWS Secret Manager. The code includes error handling, connection management, and clean resource cleanup for demonstration.

In [ ]:
import psycopg
import os
import boto3
import json

# Load environment variables from temp file
with open('.aurora_env', 'r') as f:
    for line in f:
        if line.startswith('export '):
            key, value = line.strip().replace('export ', '').split('=', 1)
            os.environ[key] = value

def get_credentials():
    """Get credentials from Secrets Manager using secret name from environment"""
    secret_name = os.getenv('SECRET_NAME')
    if not secret_name:
        raise Exception('Secret name not found. Please run the setup sections first.')
    
    secrets_client = boto3.client('secretsmanager')
    response = secrets_client.get_secret_value(SecretId=secret_name)
    secret = json.loads(response['SecretString'])
    return secret['username'], secret['password']

def get_cluster_endpoint():
    """Get cluster endpoint from environment"""
    endpoint = os.getenv('CLUSTER_ENDPOINT')
    if not endpoint:
        raise Exception('Cluster endpoint not found. Please run the setup sections first.')
    return endpoint

def test_connection():
    try:
        # Get connection details from environment
        username, password = get_credentials()
        host = get_cluster_endpoint()
        
        # Create connection
        conn = psycopg.connect(
            host=host,
            port=5432,
            dbname='mylab',
            user=username,
            password=password,
            sslmode='require'
        )
        
        # Test query
        cur = conn.cursor()
        cur.execute('SELECT current_timestamp;')
        result = cur.fetchone()
        print(f"✅ Connection successful! Server time: {result[0]}")
        
        # Clean up
        cur.close()
        conn.close()
        
    except Exception as e:
        print(f"❌ Connection failed: {str(e)}")

test_connection()

### Method 3: Using AWS CLI
Provide administrative oversight of the Aurora cluster including status monitoring and instance management. The commands display cluster health, engine information, and instance details for operational monitoring with formatted tables showing cluster status, engine version, and instance configurations.

In [ ]:
%%bash
# Source cluster info
source .aurora_env

echo "Testing cluster status with AWS CLI..."
echo "Using cluster: $AURORA_CLUSTER"

# Check cluster status
aws rds describe-db-clusters \
    --db-cluster-identifier $AURORA_CLUSTER \
    --query 'DBClusters[0].{Status:Status,Engine:Engine,Version:EngineVersion}' \
    --output table

# List instances in cluster
aws rds describe-db-instances \
    --filters Name=db-cluster-id,Values=$AURORA_CLUSTER \
    --query 'DBInstances[*].{Instance:DBInstanceIdentifier,Class:DBInstanceClass,Status:DBInstanceStatus}' \
    --output table

## 4. Connection Diagnostics

If you encounter connection issues, use these diagnostic queries.

### Common Troubleshooting Scenarios

#### 1. Connection Timeout Troubleshooting

If you encounter a connection timeout (`timeout: failed to connect to server`), run this comprehensive diagnostic check

##### Database Networking Diagnostic Script

In [ ]:
%%bash
echo "Running Connection Diagnostics..."

# Get Aurora cluster info
# Source cluster info
source .aurora_env

# Use the detected cluster ID
CLUSTER_ID=$AURORA_CLUSTER
CLUSTER_INFO=$(aws rds describe-db-clusters --db-cluster-identifier $CLUSTER_ID)
SG_ID=$(echo $CLUSTER_INFO | jq -r '.DBClusters[0].VpcSecurityGroups[0].VpcSecurityGroupId')

# Get security group info to determine VPC
SG_INFO=$(aws ec2 describe-security-groups --group-ids $SG_ID)
VPC_ID=$(echo $SG_INFO | jq -r '.SecurityGroups[0].VpcId')

echo "
1. Checking Aurora Cluster Status:"
aws rds describe-db-clusters \
    --db-cluster-identifier $CLUSTER_ID \
    --query 'DBClusters[0].{
        Status:Status,
        Engine:Engine,
        Version:EngineVersion,
        Endpoint:Endpoint,
        Port:Port,
        VpcId:"'$VPC_ID'"
    }' \
    --output table

echo "
2. Checking Network Configuration:"
echo "Security Group Rules (Port 5432):"
aws ec2 describe-security-group-rules \
    --filters Name=group-id,Values=$SG_ID \
    --query 'SecurityGroupRules[?IpProtocol==`tcp` && FromPort==`5432`].{
        Type:IsEgress,
        Protocol:IpProtocol,
        Port:FromPort,
        Source:CidrIpv4||ReferencedGroupInfo.GroupId
    }' \
    --output table

echo "Network ACLs:"
aws ec2 describe-network-acls \
    --filters Name=vpc-id,Values=$VPC_ID \
    --query 'NetworkAcls[].{
        NACL_ID:NetworkAclId,
        VPC:VpcId,
        InboundRules:Entries[?Egress==`false`].{
            RuleNumber:RuleNumber,
            Protocol:Protocol,
            PortRange:PortRange,
            Action:RuleAction
        }
    }' \
    --output table

echo "Route Tables:"
aws ec2 describe-route-tables \
    --filters Name=vpc-id,Values=$VPC_ID \
    --query 'RouteTables[].{
        RouteTable:RouteTableId,
        Routes:Routes[].{
            Destination:DestinationCidrBlock,
            Target:GatewayId||NatGatewayId||VpcPeeringConnectionId
        }
    }' \
    --output table

# Check for VPC peering connections
echo "VPC Peering Connections:"
aws ec2 describe-vpc-peering-connections \
    --filters Name=requester-vpc-info.vpc-id,Values=$VPC_ID Name=status-code,Values=active \
    --query 'VpcPeeringConnections[].{
        PeeringId:VpcPeeringConnectionId,
        RequesterVPC:RequesterVpcInfo.VpcId,
        AccepterVPC:AccepterVpcInfo.VpcId,
        AccepterCIDR:AccepterVpcInfo.CidrBlock
    }' \
    --output table

aws ec2 describe-vpc-peering-connections \
    --filters Name=accepter-vpc-info.vpc-id,Values=$VPC_ID Name=status-code,Values=active \
    --query 'VpcPeeringConnections[].{
        PeeringId:VpcPeeringConnectionId,
        RequesterVPC:RequesterVpcInfo.VpcId,
        AccepterVPC:AccepterVpcInfo.VpcId,
        RequesterCIDR:RequesterVpcInfo.CidrBlock
    }' \
    --output table

echo "
3. Checking Recent Events:"
aws rds describe-events \
    --source-identifier $CLUSTER_ID \
    --source-type db-cluster \
    --duration 60 \
    --query 'Events[*].{Time:Date,Message:Message}' \
    --output table

echo "
4. Checking Instance Health:"
aws rds describe-db-instances \
    --filters Name=db-cluster-id,Values=$CLUSTER_ID \
    --query 'DBInstances[*].{
        Instance:DBInstanceIdentifier,
        Status:DBInstanceStatus,
        PerformanceInsights:PerformanceInsightsEnabled,
        Endpoint:Endpoint.Address
    }' \
    --output table

echo "
Diagnostic Results Summary:"
echo "Check the above output for:"
echo "1. Cluster Status: Should be 'available'"
echo "2. Security Group Rules: Should allow inbound port 5432"
echo "3. Network ACLs: Should allow inbound/outbound traffic"
echo "4. Route Tables: Should have valid routes to your client"
echo "5. VPC Peering: Should show active peering if in different VPCs"
echo "6. Recent Events: Look for any relevant errors"
echo "7. Instance Health: All instances should be 'available'"

##### Common Resolution Steps

Based on the diagnostics above, here are common solutions:

1. **Security Group Issues**
Add missing security group rule:
   ```bash
   aws ec2 authorize-security-group-ingress \
       --group-id $SG_ID \
       --protocol tcp \
       --port 5432 \
       --cidr YOUR_IP_RANGE
   ```

2. **Network ACL Issues**
   - Ensure both inbound and outbound rules allow port 5432
   - Check if default deny rules are blocking traffic

3. **Routing Issues**
   - Verify private subnets have NAT Gateway route
   - Check public subnets have Internet Gateway route

4. **DNS Resolution**
Test DNS resolution:
   ```bash
   nslookup your-cluster-endpoint.region.rds.amazonaws.com
   ```

5. **Connectivity Test**
Test TCP connection:
   ```bash
   nc -zv your-cluster-endpoint.region.rds.amazonaws.com 5432
   ```

#### 2. Authentication Failed
When you see: `FATAL: password authentication failed for user`

In [ ]:
%%bash
# Source cluster and secret info
source .aurora_env

# Get credentials
SECRET_VALUE=$(aws secretsmanager get-secret-value --secret-id $SECRET_NAME --query 'SecretString' --output text)
DB_USERNAME=$(echo $SECRET_VALUE | jq -r '.username')
DB_PASSWORD=$(echo $SECRET_VALUE | jq -r '.password')

# Check user permissions and authentication settings
export PGPASSWORD=$DB_PASSWORD
psql -h $CLUSTER_ENDPOINT -p 5432 -d postgres -U $DB_USERNAME \
    -c "SELECT usename, useconfig FROM pg_user WHERE usename = current_user;" \
    -c "SELECT rolname, rolvaliduntil, rolconnlimit FROM pg_roles WHERE rolcanlogin = true;"
unset PGPASSWORD

#### 3. Too Many Connections
When you see: `FATAL: remaining connection slots are reserved`

In [ ]:
%%bash
# Source cluster and secret info
source .aurora_env

echo "Connecting database and check the current connections..."

# Get credentials
SECRET_VALUE=$(aws secretsmanager get-secret-value --secret-id $SECRET_NAME --query 'SecretString' --output text)
DB_USERNAME=$(echo $SECRET_VALUE | jq -r '.username')
DB_PASSWORD=$(echo $SECRET_VALUE | jq -r '.password')

# Check current connections
export PGPASSWORD=$DB_PASSWORD
psql -h $CLUSTER_ENDPOINT -p 5432 -d postgres -U $DB_USERNAME \
    -c "show max_connections;" \
    -c "SELECT datname, usename, count(*) as connection_count, state FROM pg_stat_activity GROUP BY datname, usename, state ORDER BY connection_count DESC;"
unset PGPASSWORD

#### Monitor Connection Metrics

Check CloudWatch metrics to monitor database connection usage over the past hour:

In [ ]:
%%bash
# Monitor connection metrics in CloudWatch
START_TIME=$(date -u -d '1 hour ago' +'%Y-%m-%dT%H:%M:%SZ' 2>/dev/null || date -u -v-1H +'%Y-%m-%dT%H:%M:%SZ')
END_TIME=$(date -u +'%Y-%m-%dT%H:%M:%SZ')

aws cloudwatch get-metric-statistics \
    --namespace AWS/RDS \
    --metric-name DatabaseConnections \
    --dimensions Name=DBClusterIdentifier,Value=aurora-demo \
    --start-time "$START_TIME" \
    --end-time "$END_TIME" \
    --period 300 \
    --statistics Maximum

## Cleanup

Clean up temporary files created during this workshop:

In [ ]:
%%bash
# Cleanup temporary files
rm -f .aurora_env
echo "✅ Temporary files cleaned up"

## Next Steps

Now that you can connect to your Aurora database, proceed to [Working with Data](2.2.2_Working_with_Data.ipynb) to learn how to:
- Create tables using Query Editor
- Load data from S3
- Run and optimize queries

## Additional Resources 📚

### Troubleshooting & Connectivity
- [Aurora Troubleshooting Guide](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.Troubleshooting.html)
- [VPC Security Best Practices](https://docs.aws.amazon.com/vpc/latest/userguide/vpc-security-best-practices.html)
- [Network Troubleshooting](https://docs.aws.amazon.com/vpc/latest/userguide/VPC_Troubleshooting.html)

### Development Tools
- [psycopg Documentation](https://www.psycopg.org/psycopg3/docs/)
- [AWS CLI RDS Commands](https://docs.aws.amazon.com/cli/latest/reference/rds/index.html)
- [AWS SDK for Python (Boto3)](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)